In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms,models
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import wandb

In [2]:
BATCH_SIZE = 64
NUM_CLASSES = 7
LEARNING_RATE = 0.001
EPOCHS_NUM = 15

wandb.init(
    project="mobilenet-skin-disease-classification",
    config={
        "batch_size": BATCH_SIZE,
        "num_classes": NUM_CLASSES,
        "learning_rate": LEARNING_RATE,
        "epochs": EPOCHS_NUM,
    }
)

wandb: Currently logged in as: tetiana-trachuk-kn-2021 (tetiana-trachuk-kn-2021-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True  # Optimizes GPU performance

# Data Transforms
augmentation_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [4]:
# Load Dataset
dataset = ImageFolder(root='C:/Diploma/classification_dataset', transform=augmentation_transforms)
train_size = int(0.85 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
test_dataset.dataset.transform = test_transforms

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)


In [5]:
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
model.to(device)

# Define Optimizer & Loss Function
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

c:\Diploma\Diploma\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Diploma\Diploma\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
for epoch in range(EPOCHS_NUM):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        labels = labels.cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels)
    
    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
    
    print(f"Epoch [{epoch+1}/{EPOCHS_NUM}], Loss: {running_loss/len(train_loader):.4f}, Acc: {acc:.4f}, Prec: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    wandb.log({
        "epoch": epoch+1, 
        "loss": running_loss/len(train_loader),  
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    })

Epoch [1/15], Loss: 0.9744, Acc: 0.6036, Prec: 0.5614, Recall: 0.5570, F1: 0.5581
Epoch [2/15], Loss: 0.7834, Acc: 0.6965, Prec: 0.6625, Recall: 0.6590, F1: 0.6603
Epoch [3/15], Loss: 0.6723, Acc: 0.7456, Prec: 0.7143, Recall: 0.7127, F1: 0.7132
Epoch [4/15], Loss: 0.6049, Acc: 0.7672, Prec: 0.7373, Recall: 0.7355, F1: 0.7362
Epoch [5/15], Loss: 0.5408, Acc: 0.7960, Prec: 0.7687, Recall: 0.7672, F1: 0.7678
Epoch [6/15], Loss: 0.4851, Acc: 0.8209, Prec: 0.7975, Recall: 0.7966, F1: 0.7969
Epoch [7/15], Loss: 0.4396, Acc: 0.8359, Prec: 0.8129, Recall: 0.8128, F1: 0.8127
Epoch [8/15], Loss: 0.3894, Acc: 0.8562, Prec: 0.8360, Recall: 0.8359, F1: 0.8357
Epoch [9/15], Loss: 0.3453, Acc: 0.8775, Prec: 0.8577, Recall: 0.8574, F1: 0.8574
Epoch [10/15], Loss: 0.3142, Acc: 0.8859, Prec: 0.8688, Recall: 0.8697, F1: 0.8691
Epoch [11/15], Loss: 0.3179, Acc: 0.8809, Prec: 0.8643, Recall: 0.8636, F1: 0.8639
Epoch [12/15], Loss: 0.2551, Acc: 0.9080, Prec: 0.8947, Recall: 0.8952, F1: 0.8949
Epoch [13/15]

In [11]:
def evaluate():
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the class with highest probability
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    # wandb.log({"test_accuracy": accuracy})

# Run Evaluation Immediately After Training
evaluate()

Test Accuracy: 75.37%


In [9]:

torch.save(model.state_dict(), "mobilenet_skin_disease_model.pth")
print("Model training complete and saved.")
wandb.finish()

Model training complete and saved.
